# DAT405 Introduction to Data Science and AI 

## Group 60: William Gerle and Alfred Kjeller, 16 hours each

## Assignment 4: Spam classification using Naïve Bayes 

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 7zip (https://www.7-zip.org/download.html) to decompress the data.


In [ ]:
#Download and extract data
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [ ]:
!ls -lah

### 1. Preprocessing: 
1.	Note that the email files contain a lot of extra information, besides the actual message. Ignore that and run on the entire text. 
2.	We don’t want to train and test on the same data. Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) **0.5p**

In [1]:
#pre-processing code here
import codecs
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Data folders
folders = ['easy_ham', 'hard_ham', 'spam']

#adding all the messages
messages = []
labels = []
for folder in folders:
    for filename in os.listdir(folder):
        with codecs.open('./{}/{}'.format(folder,filename), 'r', encoding='utf-8', errors='ignore') as fdata:
            messages.append(fdata.read().replace('\n',''))
            labels.append(folder)

#Creating dataframe with messeages and labels
df = pd.DataFrame()
df['message'] = messages
df['label'] = labels

#Printing the number of each label
n_easy = len(df[df['label'] == 'easy_ham'])
n_hard = len(df[df['label'] == 'hard_ham'])
n_spam = len(df[df['label'] == 'spam'])

print('n_ham: {}, n_easy: {}, n_hard: {}, n_spam: {}'.format(n_easy + n_hard, n_easy, n_hard, n_spam))

#The dataset is split and stratified later.

n_ham: 2801, n_easy: 2551, n_hard: 250, n_spam: 501


### 2. Write a Python program that: 
1.	Uses four datasets (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (e.g. Sklearn) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and True Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in SKlearn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem
    - Multinomial Naive Bayes  
    - Bernoulli Naive Bayes. 

In [2]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

#Classifier function where x is emails and y is labels
def classfier(x, y):
    #Train/Test split with stratify
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=0 , stratify=y)

    #Convert all email labels to spam or ham
    y_train = np.where(y_train != 'spam', 'ham', y_train)
    y_test = np.where(y_test != 'spam', 'ham', y_test)

    #Vectorize the strings
    count_vectorizer = CountVectorizer(binary='true', token_pattern=r'\w+')
    count_vectorizer.fit(x)
    emails_train = count_vectorizer.transform(x_train)
    emails_test = count_vectorizer.transform(x_test)

    results = []

    #Bernoulli Naive Bayes classifier
    bnb = BernoulliNB()
    bnb.fit(emails_train, y_train)
    results.append(['BNB', plot_confusion(bnb, emails_test, y_test, 'Bernoulli Naive Bayes')])

    #Multinomial Naive Bayes classifier
    mnb = MultinomialNB()
    mnb.fit(emails_train, y_train)
    results.append(['MNB', plot_confusion(mnb, emails_test, y_test, 'Multinomial Naive Bayes')])

    return results

#Plots/prints accuracy for classifiers, returns array with true positive/negative rates and total accuracy
def plot_confusion(model, x_values, y_values, title):
    y_pred = model.predict(x_values)
    res = metrics.confusion_matrix(y_values, y_pred)
    tpr = round((res.item(0) / (res.item(0) + res.item(1))) * 100, 2)
    tnr = round((res.item(3) / (res.item(2) + res.item(3))) * 100, 2)
    acc = round(((res.item(0) + res.item(3)) / (res.item(0) + res.item(1) + res.item(2) + res.item(3))) * 100, 2)
    print("{} True positive rate: {}%  True negative rate: {}%  Total accuracy: {}%".format(title, tpr, tnr, acc))

    #Set plots to True to plot a confusion matrix.
    plots = False
    if plots:
        plt.rcParams["figure.figsize"] = (10,10)
        plt.rcParams.update({'font.size': 20})
        matrix = metrics.plot_confusion_matrix(model, x_values, y_values, cmap=plt.cm.Blues)
        plt.title(title)
        plt.show(matrix)
        plt.show()

    return [tpr, tnr, acc]

**a.** Explain how the classifiers differ. What different interpretations do they have? **1p** 

The Bernoulli Naive Bayes (BNB) classifier does only consider if a feature occurs or if it does not. The Multinomial Naive Bayes (MNB) classifier functions similarly but this classifier also considers how often a feature occurs. For example if the word "invest" is considered a spam word, the BNB classifier only considers if the word exists in an email or not, whilst the MNB classifier also considers how many times it occurs. Considering this, in our opinion, the BNB classifier is more appropriate when the features are binary, which is not the case in this assignment.

### 3.Run your program on 
-	Spam versus easy-ham 
-	Spam versus (hard-ham + easy-ham). 
-   Discuss your results **2.5p** 

In [3]:
#Code to report results here
print('Spam versus easy-ham:')
easy = df[df['label'] != 'hard_ham']
easy_3 = classfier(easy.message.to_numpy(), easy.label.to_numpy())

print('\nSpam versus (hard-ham + easy-ham):')
hard_3 = classfier(df.message.to_numpy(), df.label.to_numpy())

Spam versus easy-ham:
Bernoulli Naive Bayes True positive rate: 99.48%  True negative rate: 38.67%  Total accuracy: 89.52%
Multinomial Naive Bayes True positive rate: 99.87%  True negative rate: 79.33%  Total accuracy: 96.51%

Spam versus (hard-ham + easy-ham):
Bernoulli Naive Bayes True positive rate: 98.22%  True negative rate: 20.0%  Total accuracy: 86.38%
Multinomial Naive Bayes True positive rate: 99.64%  True negative rate: 71.33%  Total accuracy: 95.36%


Firstly, one interesting observation is that the accuracies of the models are lower when running the program on spam versus (hard-ham + easy-ham) compared to running the program on spam versus easy-ham. The reason for this is that hard-ham emails are more difficult to predict, which means that including them will probably lead to less accurate predictions in total.

Secondly, it is interesting that the BNB classifiers have lower accuracies compared to the MNB classifiers. The reason for this is that the BNB classifier only considers if a feature occurs or not, while the MNB classifier also considers how often a feature occurs. Therefore, in this case, where the features are not binary, the MNB classifier is more accurate. For example, if "invest" is considered to be a word that increases the probability of an email being spam. Then, consider a spam email where this word occurs four times. In this example, the BNB model will only register that this email contains a spam word, while the MNB classifier will take into consideration that it occurs four times and is therefore probably more likely to predict the email as spam. 

Thirdly, all the models have a higher true positive rate compared to true negative rate. A probable reason behind this is the distribution among classes in the training data. The dataset contains 2801 ham emails and only 501 spam emails. Because of this does the classifiers have a lot less information about spam emails which results in less accurate predictions for these types of emails. 

Lastly, perhaps it is more important to have a high true positive rate rather than a high true negative rate since reasonably it is less disturbing to receive some spam mails in your ordinary inbox compared to having ham emails ending up in your spam inbox. Nonetheless, this could be considered to be subjective and could differ between individuals.

### 4.	To avoid classification based on common and uninformative words it is common to filter these out. 

**a.** Argue why this may be useful. Try finding the words that are too common/uncommon in the dataset. **1p** 

**b.** Use the parameters in Sklearn’s `CountVectorizer` to filter out these words. Update the program from point 3 and run it on your data and report and discuss your results. You have two options to do this in Sklearn: either using the words found in part (a) or letting Sklearn do it for you. Argue for your decision-making. **1p** 

**Text part a.**

It is typical to filter out common and uncommon words before using a classifier. Regarding the common words, this is useful since these words probably do not help the model in determining if an email is spam or ham considering that they exist regularly in both types. Regarding the uncommon words, for example if a word only occurs in one email in the training data, this will largely increase the probability that all emails in the test data that contains this word is labeled the same as the email in the training data. Therefore, by filtering out these types of words, the model avoids classifications on these words and becomes more accurate. 

In our opinion the definition of a common or uncommon word is based on the number of emails in which it occurs rather than the total word frequency in the dataset. In our opinion this definition is more accurate and will hopefully be the best choice for improving the model. Regarding the filter limits, filtering out too many words could lead to insufficient training data for the classifier. On the other hand, filtering out too few words could result in no improvements to the classifier. Therefore, in order to achieve a high accuracy and a good balance between filtering out too much versus too little, the choice was made to filter out words that appear in less than three emails and words that appear in more than 90 percent of the emails.

In [4]:
#Code part a. Finding the common/uncommon words

#Our method of filtering out common/uncommon words
from collections import Counter
import re

counts = Counter()
words = re.compile(r'\w+')

for sentence in df.message:
    counts.update(list(dict.fromkeys(words.findall(sentence.lower()))))

bad_words = []

for w in counts:
    if counts[w] > (n_easy + n_hard + n_spam) * 0.90:
        bad_words.append(w)
    elif counts[w] < 3:
        bad_words.append(w)
print('Number of common/uncommon words with our method: {}'.format(len(bad_words)))

#Using CounterVectorizer to filter out common/uncommon words
vectorizer = CountVectorizer(binary='true', max_df=0.90, min_df=3, token_pattern=r'\w+')
vectorizer.fit(df.message)
print('Number of common/uncommon words with CountVectorizer: {}'.format(len(vectorizer.stop_words_)))

#bad_words and vectorizer.stop_words_ contains the common/uncommon words.

Number of common/uncommon words with our method: 96712
Number of common/uncommon words with CountVectorizer: 96712


**Text part b**

When running our model after removing the common/uncommon words, the true positive rate for all classifiers is slightly lower than previously. However, all true negative rates are much higher than previously which results in a higher total accuracy after removing common/uncommon words. As discussed in part a, it is reasonable that the total performance is increased since the classifiers avoids classifying on common and uncommon words which are words that could hinder a high model performance. 

However, as earlier mentioned, perhaps it is more important to have a high true positive rate rather than a high true negative rate since reasonably it is less disturbing to receive some spam mails in your ordinary inbox compared to having ham emails ending up in your spam inbox. Nonetheless, this could be considered to be subjective and could differ between individuals.

Regarding the method for filtering out words, both the option of using the words found in part (a) and the option to let Sklearn do it for us were tested. Both options resulted in identical results, since both options filter out the same words. Therefore, the choice was made to use the built in function in CountVectorizer since this provided a sleeker code in our opinion.

In [5]:
#Code part b. 

#Classifier function where x is emails and y is labels
def classfier_stopwords(x, y):
    #Train/Test split with stratify
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=0 , stratify=y)

    #Convert all email labels to spam or ham
    y_train = np.where(y_train != 'spam', 'ham', y_train)
    y_test = np.where(y_test != 'spam', 'ham', y_test)        

    #Vectorize the strings and filtering out common/uncommon words
    count_vectorizer = CountVectorizer(binary='true', max_df=0.90, min_df=3, token_pattern=r'\w+')
    count_vectorizer.fit(x)
    emails_train = count_vectorizer.transform(x_train)
    emails_test = count_vectorizer.transform(x_test)
    
    results = []
    
    #Bernoulli Naive Bayes classifier
    bnb = BernoulliNB()
    bnb.fit(emails_train, y_train)
    results.append(['BNB', plot_confusion(bnb, emails_test, y_test, 'Bernoulli Naive Bayes')])

    #Multinomial Naive Bayes classifier
    mnb = MultinomialNB()
    mnb.fit(emails_train, y_train)
    results.append(['MNB', plot_confusion(mnb, emails_test, y_test, 'Multinomial Naive Bayes')])
    return results


print('Spam versus easy-ham with stop words:')
easy = df[df['label'] != 'hard_ham']
easy_4 = classfier_stopwords(easy.message.to_numpy(), easy.label.to_numpy())

print('\nSpam versus (hard-ham + easy-ham) with stop words:')
hard_4 = classfier_stopwords(df.message.to_numpy(), df.label.to_numpy())

Spam versus easy-ham with stop words:
Bernoulli Naive Bayes True positive rate: 99.09%  True negative rate: 97.33%  Total accuracy: 98.8%
Multinomial Naive Bayes True positive rate: 99.61%  True negative rate: 98.67%  Total accuracy: 99.45%

Spam versus (hard-ham + easy-ham) with stop words:
Bernoulli Naive Bayes True positive rate: 96.2%  True negative rate: 93.33%  Total accuracy: 95.76%
Multinomial Naive Bayes True positive rate: 97.5%  True negative rate: 97.33%  Total accuracy: 97.48%


### 5. Eeking out further performance
**a.**  Use a lemmatizer to normalize the text (for example from the `nltk` library). For one implementation look at the documentation ([here](https://scikit-learn.org/stable/modules/feature_extraction.html#customizing-the-vectorizer-classes)). Run your program again and answer the following questions: 
  - Why can lemmatization help?
  -	Does the result improve from 3 and 4? Discuss. **1.5p** 






**Text part a**
- Lemmatization resolves words to their dictionary form, also called lemmas. This requires knowledge about the structure of the used language, which is why the WordNet database is used. For example, the lemmatization algorithm knows that the word *better* is derived from the word *good*. Therefore, even if the word *better* never occurred in the training data, the classifier can still handle a situation where this word shows up in test data. In other words, by resolving words to their dictionary form the classifier could become more accurate.
  
- The classifiers from 3 does still have the best true positive rates, while the classifier from 4 and the classifier using lemmatization is slightly lower. The most interesting differences between the models are in the true negative rates, where the classifiers from 3 have significantly lower rates compared to the classifiers from 4 and the ones using lemmatization. 

- When comparing the classifiers from 4 and the classifiers using lemmatization, the rates are very similar. Even though the accuracy stays essentially the same when introducing lemmatization, the classifier will probably be better. The reason behind this is that the classifier with lemmatization can handle more words and therefore have a higher generalizability. An example of this is explained above with the words *good* and *better*.

In [6]:
#Code part a

#Imports and downloading nltk_data
import nltk         
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('wordnet')

#Lemmanizer class
class LemmaTokenizer:
    def __init__(self):
         self.wnl = WordNetLemmatizer()
         self.tokenizer = nltk.RegexpTokenizer(r"\w+")
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in self.tokenizer.tokenize(doc)]
        
#Classifier function where x is emails and y is labels
def classfier_lemmatization(x, y):
    #Train/Test split with stratify
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=0 , stratify=y)

    #Convert all email labels to spam or ham
    y_train = np.where(y_train != 'spam', 'ham', y_train)
    y_test = np.where(y_test != 'spam', 'ham', y_test)        

    #Vectorize the strings and lemmanize
    count_vectorizer = CountVectorizer(binary='true', max_df=0.90, min_df=3, tokenizer=LemmaTokenizer())
    count_vectorizer.fit(x)
    emails_train = count_vectorizer.transform(x_train)
    emails_test = count_vectorizer.transform(x_test)
    
    results = []

    #Bernoulli Naive Bayes classifier
    bnb = BernoulliNB()
    bnb.fit(emails_train, y_train)
    results.append(['BNB', plot_confusion(bnb, emails_test, y_test, 'Bernoulli Naive Bayes')])

    #Multinomial Naive Bayes classifier
    mnb = MultinomialNB()
    mnb.fit(emails_train, y_train)
    results.append(['MNB', plot_confusion(mnb, emails_test, y_test, 'Multinomial Naive Bayes')])
    return results


print('Spam versus easy-ham with stop words and lemmatization:')
easy = df[df['label'] != 'hard_ham']
easy_5 = classfier_lemmatization(easy.message.to_numpy(), easy.label.to_numpy())

print('\nSpam versus (hard-ham + easy-ham) with stop words and lemmatization:')
hard_5 = classfier_lemmatization(df.message.to_numpy(), df.label.to_numpy())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Asus\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Spam versus easy-ham with stop words and lemmatization:
Bernoulli Naive Bayes True positive rate: 99.09%  True negative rate: 96.67%  Total accuracy: 98.69%
Multinomial Naive Bayes True positive rate: 99.61%  True negative rate: 98.67%  Total accuracy: 99.45%

Spam versus (hard-ham + easy-ham) with stop words and lemmatization:


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Bernoulli Naive Bayes True positive rate: 96.2%  True negative rate: 94.0%  Total accuracy: 95.86%
Multinomial Naive Bayes True positive rate: 97.86%  True negative rate: 98.0%  Total accuracy: 97.88%


**b.** The split of the data set into a training set and a test set can lead to very skewed results. Why is this, and do you have suggestions on remedies? 
 What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages?  **1p** 

When splitting the data, the distribution of different types of emails in the respective sets could be different. If for example, the training set were mostly spam messages while the test set were mostly ham messages, the classifier would most likely end up with a low accuracy. The reason for this is that the model has almost exclusively been trained on spam messages and is therefore only good at predicting those kinds. Consequently, when testing the classifier on ham messages the classifier will not make accurate predictions since it does not recognize this type. For reference, this example is similar to creating a model with the goal to be able to differentiate between a cat and a dog, but only use pictures of cats to train the model.

To solve this issue, stratify could be used which makes sure that the distribution of different types in the dataset is kept when splitting into training set and test set. Another solution is to use undersampling and/or oversampling which means to delete data points from the majority class and/or duplicate data points from the minority class.

**c.** Re-estimate your classifier using `fit_prior` parameter set to `false`, and answer the following questions:
  - What does this parameter mean?
  - How does this alter the predictions? Discuss why or why not. **0.5p** 

- The fit_prior parameter determines whether to learn class prior probabilities or not. This parameter is by default set to true, which means that class prior probabilities will be learned. If set to false, uniform prior probabilities will be used.

- In this case the results of the classifiers did barely change when setting fit_prior to false. However, this parameter is still important. If set to True, the classifier will be biased based on the distribution of the classes in the training data. In this dataset there are more ham emails compared to spam emails which means that the classifier will be biased towards predicting an email as ham without considering the content. If set to False, a uniform prior will be used instead which removes the bias from the classifier. If the ratio between classes is known, it could be beneficial to have fit_prior set to True to benefit from this ratio knowledge. However, this could lead to overfitting and poor performance on test data if the ratio of classes is not the same as in the training data. Therefore, in our opinion, it is probably more often beneficial to use uniform priors for the sake of generalizability of the classifiers.

In [7]:
#Classifier function where x is emails and y is labels
def classfier_lemmatization_fit_prior(x, y):
    #Train/Test split with stratify
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=0 , stratify=y)

    y_train = np.where(y_train != 'spam', 'ham', y_train)
    y_test = np.where(y_test != 'spam', 'ham', y_test)        

    #Convert all email labels to spam or ham
    count_vectorizer = CountVectorizer(binary='true', max_df=0.90, min_df=3, tokenizer=LemmaTokenizer())
    count_vectorizer.fit(x)
    emails_train = count_vectorizer.transform(x_train)
    emails_test = count_vectorizer.transform(x_test)
    
    results = []

    #Bernoulli Naive Bayes classifier
    bnb = BernoulliNB(fit_prior=False)
    bnb.fit(emails_train, y_train)
    results.append(['BNB', plot_confusion(bnb, emails_test, y_test, 'Bernoulli Naive Bayes')])

    #Multinomial Naive Bayes classifier
    mnb = MultinomialNB(fit_prior=False)
    mnb.fit(emails_train, y_train)
    results.append(['MNB', plot_confusion(mnb, emails_test, y_test, 'Multinomial Naive Bayes')])
    return results


print('Spam versus easy-ham with stop words and lemmatization:')
easy = df[df['label'] != 'hard_ham']
easy_6 = classfier_lemmatization_fit_prior(easy.message.to_numpy(), easy.label.to_numpy())

print('\nSpam versus (hard-ham + easy-ham) with stop words and lemmatization:')
hard_6 = classfier_lemmatization_fit_prior(df.message.to_numpy(), df.label.to_numpy())

Spam versus easy-ham with stop words and lemmatization:


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Bernoulli Naive Bayes True positive rate: 99.09%  True negative rate: 97.33%  Total accuracy: 98.8%
Multinomial Naive Bayes True positive rate: 99.61%  True negative rate: 98.67%  Total accuracy: 99.45%

Spam versus (hard-ham + easy-ham) with stop words and lemmatization:


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Bernoulli Naive Bayes True positive rate: 96.2%  True negative rate: 94.0%  Total accuracy: 95.86%
Multinomial Naive Bayes True positive rate: 97.5%  True negative rate: 98.0%  Total accuracy: 97.58%


**d.** The python model includes smoothing (`alpha` parameter ), explain why this can be important. 
  - What would happen if in the training data set the word 'money' only appears in spam examples? What would the model predict about a message containing the word 'money'? Does the prediction depend on the rest of the message and is that reasonable? Explain your reasoning  **1p** 

- Smoothing handles the problem that can arise when a word in the test data appears that was not present in the training data. In such cases, if smoothing was not included in the model, the probability of that email being either spam or ham would both end up being set to zero, since the multiplications in the numerator will include a zero.  Since the training set is supposed to represent reality, it is problematic to predict such an email as neither spam or ham just because some words are not present in the training data. Smoothing solves this problem by giving missing words a small non-zero probability for both classes. Therefore, by solving this potential problem, smoothing contributes to a better classifier.

- If the word 'money' only appears in spam emails in the training data, the probability for an email containing this word being spam, will be increased by this fact. Nonetheless, the probability still depends on the rest of the message. However, if smoothing is not used, the probability of the word being ham would be zero since 'money' never appeared in such an email in the training data. Therefore, the prediction will always be spam and consequently it could be viewed as the prediction is solely based on the word 'money', since the probability of spam will always be higher than ham (which is zero). However, if smoothing is used, as explained before, the probability of ham will not end up being set to zero and therefore the classifier will still depend on the rest of the message, which in our opinion is more reasonable.

In [8]:
#"Pretty" code to print all results
easy_results = [easy_3[0]] + [easy_4[0]] + [easy_5[0]] + [easy_6[0]] + [easy_3[1]] + [easy_4[1]] + [easy_5[1]] + [easy_6[1]]
hard_results = [hard_3[0]] + [hard_4[0]] + [hard_5[0]] + [hard_6[0]] + [hard_3[1]] + [hard_4[1]] + [hard_5[1]] + [hard_6[1]]

print("All results:")
i = [3,4,5,6,3,4,5,6]
n=0
print('Spam versus easy-ham:')
for res in easy_results:
    print("Q{} {} True positive rate: {}%  True negative rate: {}%  Total accuracy: {}%".format(i[n],res[0], res[1][0], res[1][1], res[1][2]))
    n += 1

print('\nSpam versus (easy + hard_ham):')
n=0
for res in hard_results:
    print("Q{} {} True positive rate: {}%  True negative rate: {}%  Total accuracy: {}%".format(i[n],res[0], res[1][0], res[1][1], res[1][2]))
    n += 1

All results:
Spam versus easy-ham:
Q3 BNB True positive rate: 99.48%  True negative rate: 38.67%  Total accuracy: 89.52%
Q4 BNB True positive rate: 99.09%  True negative rate: 97.33%  Total accuracy: 98.8%
Q5 BNB True positive rate: 99.09%  True negative rate: 96.67%  Total accuracy: 98.69%
Q6 BNB True positive rate: 99.09%  True negative rate: 97.33%  Total accuracy: 98.8%
Q3 MNB True positive rate: 99.87%  True negative rate: 79.33%  Total accuracy: 96.51%
Q4 MNB True positive rate: 99.61%  True negative rate: 98.67%  Total accuracy: 99.45%
Q5 MNB True positive rate: 99.61%  True negative rate: 98.67%  Total accuracy: 99.45%
Q6 MNB True positive rate: 99.61%  True negative rate: 98.67%  Total accuracy: 99.45%

Spam versus (easy + hard_ham):
Q3 BNB True positive rate: 98.22%  True negative rate: 20.0%  Total accuracy: 86.38%
Q4 BNB True positive rate: 96.2%  True negative rate: 93.33%  Total accuracy: 95.76%
Q5 BNB True positive rate: 96.2%  True negative rate: 94.0%  Total accuracy: 

### What to report and how to hand in.

- You will need to clearly report all results in the notebook in a clear and appropriate way, either using plots or code output (f.x. "print statements"). 
- The notebook must be reproducible, that means, we must be able to use the `Run all` function from the `Runtime` menu and reproduce all your results. **Please check this before handing in.** 
- Save the notebook and share a link to the notebook (Press share in upper left corner, and use `Get link` option. **Please make sure to allow all with the link to open and edit.**
- Edits made after submission deadline will be ignored, graders will recover the last saved version before deadline from the revisions history.
- **Please make sure all cells are executed and all the output is clearly readable/visible to anybody opening the notebook.**